# CEC1702 BIGI GLITCHING

In [7]:
#adjust notebook size 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 1. Connecting to ChipWhisperer and CEC1702
### Import CW and default setup 

In [8]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER='SS_VER_1_1'
import chipwhisperer as cw
print(".Import DONE")
scope = cw.scope()
print("..Scope set")
target = cw.target(scope, cw.targets.SimpleSerial) #cw.targets.SimpleSerial can be omitted
print("...Target set")
scope.default_setup()
print("....Scope setup done")

.Import DONE
..Scope set
...Target set
....Scope setup done


### Scope setup

#### ⚠ Ensure J3 on UFO Board routines CLKFB to HS1/IN ( the lowest jumper!) ⚠
****
#### CEC1702 must be programmed externaly using Flash Programmer
****

In [9]:
import time

def setup_scope_for_glitching():
    scope.adc.basic_mode = "rising_edge"
    scope.clock.clkgen_src = "system"    #ChipWhisperer-Lite CLK
    scope.clock.adc_src = "clkgen_x1"    #Only for ADC
    scope.clock.freq_ctr_src = "clkgen"  #ChipWhisperer-Lite CLK, not neccessary
    scope.clock.clkgen_freq = 48_000_000
    time.sleep(1.5)
    scope.io.hs2 = None
    
setup_scope_for_glitching()  
print(scope.clock)
print("\033[1m***********************************************\033[0m")
print(f"\033[1mSample rate = {scope.clock.freq_ctr} Hz")        
print(f"Target baudrate = {target.baud}\033[0m")

adc_src       = clkgen_x1
adc_phase     = 0
adc_freq      = 48000000
adc_rate      = 48000000.0
adc_locked    = True
freq_ctr      = 48000000
freq_ctr_src  = clkgen
clkgen_src    = system
extclk_freq   = 10000000
clkgen_mul    = 2
clkgen_div    = 4
clkgen_freq   = 48000000.0
clkgen_locked = True

***********************************************
Sample rate = 48000000 Hz
Target baudrate = 38400


#### Function definition for device rebooting/reseting

In [10]:
import time
def reboot_flush():            
    scope.io.nrst = False
    time.sleep(0.5)
    scope.io.nrst = "high_z"
    time.sleep(0.3)
    #Flush garbage too
    target.flush()

reboot_flush()
print(scope.clock)

adc_src       = clkgen_x1
adc_phase     = 0
adc_freq      = 48000000
adc_rate      = 48000000.0
adc_locked    = True
freq_ctr      = 48000000
freq_ctr_src  = clkgen
clkgen_src    = system
extclk_freq   = 10000000
clkgen_mul    = 2
clkgen_div    = 4
clkgen_freq   = 48000000.0
clkgen_locked = True



## 2. Test CEC1702 computation and answer

### A . When MCU answer is only r-0x00/0x01

In [11]:
import time
scope.clock.adc_src = "clkgen_x1"
target.flush()
scope.arm()

tak = time.perf_counter()
target.simpleserial_write('b', bytearray([]))
ret = scope.capture()
val = target.simpleserial_read_witherrors('r', 1, timeout=0)
tik = time.perf_counter()
print(f"ModRed in {tik - tak:0.4f} seconds")

valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(f"output = {val}")
print(f"scope.adc.trig_count = {scope.adc.trig_count}")

ModRed in 0.0211 seconds
output = {'valid': True, 'payload': CWbytearray(b'00'), 'full_response': 'r00\n', 'rv': 0}
scope.adc.trig_count = 3927


## 3.  Setting up the glitch module for VOLTAGE-GLITCHING

In [12]:
scope.glitch.clk_src = "clkgen" # set glitch input clock - 48 MHz
scope.glitch.output = "glitch_only"
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.io.glitch_hp = True
scope.io.glitch_lp = True

In [13]:
import time

def reboot_UFO_board(ext_offset):
    scope.io.target_pwr = False
    time.sleep(0.1)
    print(f"Rebooted:EO = {scope.glitch.ext_offset}")
    scope.io.target_pwr = True
    time.sleep(0.1)
    
def timer(start,end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds)

In [27]:
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

IntText(value=0, description='false count:', disabled=True)

IntText(value=0, description='UFO_reset count:', disabled=True)

FloatSlider(value=45.0, continuous_update=False, description='width setting:', disabled=True, max=49.0, min=45…

FloatSlider(value=-44.0, continuous_update=False, description='offset setting:', disabled=True, max=-40.0, min…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=3929.0, …

## 4. Glitching loop

 ### A. Glitching only getting 0x00/0x01 response

In [40]:
import matplotlib.pylab as plt
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
import struct
from math import gcd
import gmpy2
from gmpy2 import mpz
import time

succesful_glitches = []
resets = []
# for repeat_param in range(10, 12):
gc = glitch.GlitchController(groups=["success", "reset", "normal", "false", "UFO_reset"], parameters=["width", "offset", "ext_offset", "repeat"])
gc.display_stats()

g_step = 1
OFFSET_START =    0
OFFSET_END   = 3929
REPEAT_MIN   =   7
REPEAT_MAX   =   9
gc.set_range("width", 47, 49)
gc.set_range("offset", -44, -40)
gc.set_range("ext_offset", OFFSET_START, OFFSET_END)
gc.set_range("repeat", REPEAT_MIN, REPEAT_MAX)

gc.set_global_step(g_step)
scope.adc.timeout = 1
reboot_flush()

#some magic numbers here don't know why not corresponding with ms according a documentati
enc_timeout = 2000

total_success = 0
total_reset = 0
total_normal = 0
total_duration = 0
total_false_success = 0
total_UFO_reset = 0
resets_in_series = 0
loop_start_time = time.time()
time_to_proccess_1000_offsets_old = time.time()

for glitch_setting in gc.glitch_values():
    scope.glitch.width      = glitch_setting[0]
    scope.glitch.offset     = glitch_setting[1]
    scope.glitch.ext_offset = glitch_setting[2]
    scope.glitch.repeat     = glitch_setting[3]

    for i in range(1):
        if resets_in_series > 3:
            reboot_UFO_board(glitch_setting[2])
            gc.add("UFO_reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat))
            resets_in_series = 0
            total_UFO_reset += 1

        tak = time.perf_counter() 
        target.flush()
        scope.adc.timeout = 0.5
        scope.arm()
        target.simpleserial_write('b', bytearray([]))
        ret = scope.capture()

        if ret:
            print('Timeout happened during acquisition')
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat))
            resets.append([scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat])
            total_reset += 1
            resets_in_series += 1
            reboot_flush()
            continue


        output = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10) #For loop check
        scope.io.glitch_hp = False
        scope.io.glitch_hp = True
        scope.io.glitch_lp = False
        scope.io.glitch_lp = True

        if output['valid'] is False:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat))
            resets.append([scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat])
            total_reset += 1
            resets_in_series += 1
            reboot_flush()
            continue
        else:
            if output['payload'] == bytearray([1]): #for loop check
                gc.add("success", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat))
                tik = time.perf_counter()
                total_success += 1
                total_duration += (tik - tak)
                resets_in_series = 0
                succesful_glitches.append([scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat])
            else:
                total_normal += 1
                resets_in_series = 0
                gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, scope.glitch.repeat))

loop_end_time = time.time()

print(f"- total_success   = {total_success}")
print(f"- total_normal    = {total_normal}")
print(f"- total_reset     = {total_reset}")
print(f"- total_UFO_reset = {total_UFO_reset}")
print(f"- duration        = {timer(loop_start_time, loop_end_time)}")

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

IntText(value=0, description='false count:', disabled=True)

IntText(value=0, description='UFO_reset count:', disabled=True)

...

Timeout happened during acquisition


Timeout happened during acquisition


Timeout happened during acquisition


Timeout happened during acquisition
Rebooted:EO = 3830


Timeout happened during acquisition
- total_success   = 85
- total_normal    = 166821
- total_reset     = 9944
- total_UFO_reset = 419
- duration        = 04:14:29.49


In [1]:
scope.dis()
target.dis()

NameError: name 'scope' is not defined

In [2]:
text_arr = []

w = 1
o = 20
o_e = 222

text_arr.append([w, o, o_e])
text_arr.append([o, w, o_e])

print(text_arr)

[[1, 20, 222], [20, 1, 222]]


In [41]:
#Save the results
FILE_NAME = "bigi_testing_2"

In [42]:
%%sh -s "$FILE_NAME"
cd CEC_GLITCHING
touch offsets_$1.txt
touch resets_$1.txt
echo "files created"

files created


In [43]:
import numpy as np
np_succesful_glitches = np.array(succesful_glitches)
np.savetxt("CEC_GLITCHING/offsets_" + FILE_NAME + ".txt", np_succesful_glitches, fmt='%1.3f', delimiter = ", ")

np_resets = np.array(resets)
np.savetxt("CEC_GLITCHING/resets_" + FILE_NAME + ".txt", np_resets, fmt='%1.3f', delimiter = ", ")

print("files filled with data")



# with open("CEC_GLITCHING/info_" + FILE_NAME + ".txt", 'w') as f:
#     f.write(f'OFFSET_START = {OFFSET_START}\n')
#     f.write(f'OFFSET_END   = {OFFSET_END}  \n')
#     f.write(f'-----------------------------\n')
#     f.write(f'N   = {n}\n')
#     f.write(f'CT  = {c}\n')    

files filled with data


# repeat == 1

Done glitching:
- total_success   = 0
- total_normal    = 3931
- total_reset     = 0
- total_UFO_reset = 0
- total_false_s   = 0
- offset count    = 3931
- duration        = 00:02:06.36

# repeat == 10

In [34]:
print(succesful_glitches)

[[46.09375, -44.140625, 153, 10], [46.09375, -44.140625, 251, 11], [46.09375, -44.140625, 393, 11], [46.09375, -44.140625, 484, 11], [46.09375, -44.140625, 558, 11], [46.09375, -44.140625, 1190, 11], [46.09375, -44.140625, 1199, 11], [46.09375, -44.140625, 1261, 11], [46.09375, -44.140625, 1383, 12], [46.09375, -44.140625, 1425, 12], [46.09375, -44.140625, 1427, 11], [46.09375, -44.140625, 1598, 11], [46.09375, -44.140625, 1619, 12], [46.09375, -44.140625, 1750, 12], [46.09375, -44.140625, 1751, 11], [46.09375, -44.140625, 1773, 12], [46.09375, -44.140625, 1775, 12], [46.09375, -44.140625, 1777, 12], [46.09375, -44.140625, 1779, 11], [46.09375, -44.140625, 1800, 11], [46.09375, -44.140625, 1824, 12], [46.09375, -44.140625, 1999, 11], [46.09375, -44.140625, 2484, 12], [46.09375, -44.140625, 2571, 11], [46.09375, -44.140625, 2619, 11], [46.09375, -44.140625, 2761, 12], [46.09375, -44.140625, 2782, 11], [46.09375, -44.140625, 2835, 12], [46.09375, -44.140625, 2854, 11], [46.09375, -44.140

# repeat == 11

# repeat == 20

## All repeats

Done glitching (repeat = 10, 9,... 1):
- total_success   = 0
- total_normal    = 3931
- total_reset     = 0
- total_UFO_reset = 0
- total_false_s   = 0
- offset count    = 3931
- duration        = 00:02:04.33
        
Done glitching (repeat = 11):
- total_success   = 6
- total_normal    = 3915
- total_reset     = 10
- total_UFO_reset = 0
- total_false_s   = 0
- offset count    = 3931
- duration        = 00:02:13.37

Done glitching (repeat = 12):
- total_success   = 32
- total_normal    = 2803
- total_reset     = 1096
- total_UFO_reset = 35
- total_false_s   = 0
- offset count    = 3931
- duration        = 00:18:55.30
        
Done glitching (repeat = 13):
- total_success   = 18
- total_normal    = 916
- total_reset     = 2997
- total_UFO_reset = 468
- total_false_s   = 0
- offset count    = 3931
- duration        = 00:49:32.42

Done glitching (repeat = 14):
- total_success   = 4
- total_normal    = 162
- total_reset     = 3765
- total_UFO_reset = 875
- total_false_s   = 0
- offset count    = 3931
- duration        = 01:03:20.75